# **Tercer Entrega - Sistemas de Recomendación**

En esta entrega vamos a trabajar con un sistema de recomendación.

Vamos a aprovechar la competencia de Telecom ya que los datos son reales y están buenos para practicar.

En el siguiente repositorio pueden encontrar el significado de cada columna de los datasets:
https://github.com/Datathon2021/Recomendador

# Consigna:

- Dividir set en train y test. Tomar como train los datos hasta el 1 de marzo de 2021. Desde el 1ro de marzo en adelante, reservar para test.
- Desarrollar un recomendador. El recomendador debe ser capaz de generar recomendaciones para TODOS los usuarios (incluyendo los cold start que no tengan visualizaciones en el set de train). Generar 20 recomendaciones por usuario.
- Las recomendaciones tienen que ser para cada account_id y hay que recomendar content_id (NO asset_id). Pueden encontrar esto en el repositorio de la competencia.
- Los contenidos que recomienden, no tienen que haber sido vistos previamente por los usuarios.
- Evaluarlo con MAP.

### Recomendaciones:
- En este caso no tenemos ratings explícitos, deben generar ustedes estos ratings mediante algún criterio. Lo más simple podría ser utilizar ratings binarios.
- La columna **end_vod_date**: "fecha de finalización de la disponibilidad del activo en la plataforma" puede llegar a serles útil. ¿Tiene sentido recomendar algo que no va a estar disponible en el set de test?.


### Datos:
- https://drive.google.com/file/d/1QjbqJvU0PTClEhReOyzr96BS-HpyyUKT/view?usp=sharing
- https://drive.google.com/file/d/1ih6SZc44w8FXdFKQJ2l-aqEWFpfceGuN/view?usp=sharing




#**Resolución**

Montamos Drive

In [1]:
from google.colab import drive # La usamos para montar nuestra unidad de Google Drive
drive.mount('/content/drive') # Montamos nuestra unidad de Google Drive

Mounted at /content/drive


In [2]:
import pandas as pd
from datetime import datetime

Leemos los datset: metadata y train

In [3]:
metadata= pd.read_csv("/content/drive/MyDrive/Data_Science_2021/Entregas/Entrega_3/metadata.csv", sep= ';',parse_dates=['start_vod_date','end_vod_date'])
train= pd.read_csv("/content/drive/MyDrive/Data_Science_2021/Entregas/Entrega_3/train.csv",parse_dates=['tunein', 'tuneout'])

In [4]:
metadata.head()

,asset_id,content_id,title,reduced_title,episode_title,show_type,released_year,country_of_origin,category,keywords,description,reduced_desc,cast_first_name,credits_first_name,run_time_min,audience,made_for_tv,close_caption,sex_rating,violence_rating,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
0,15188,0.0,Ep:17 Tiempos Compulsivos,Tiempos_Compul_E17,Episodio 17,Serie,2012.0,AR,Drama,"Trastornos,Médicos,Tragicómica,Telenovela,Enfe...",Esteban recibe una noticia inesperada: va a se...,Esteban recibe una noticia inesperada: va a se...,"Rodrigo De la Serna, Paola Krum, Fernán Mirás,...",Daniel Barone,48.0,General,N,N,N,N,N,N,N,N,N,N,2017-12-01T10:18:15.0Z,2019-01-26T06:37:18.0Z,2017-12-01 00:00:00+00:00,2020-12-01 23:59:59+00:00
1,24940,1.0,7 Cajas,7_Cajas,NaN,Película,2012.0,PY,Suspenso/Acción,"Latinoamérica,Pobreza,Crimen,Pandillas",Víctor trabaja como carretillero. Un día recib...,"Trabajando como carretillero, Víctor recibe un...","Celso Franco, Víctor Sosa, Lali Gonzalez",Juan Carlos Maneglia,105.0,General,N,N,N,N,N,N,N,Y,N,N,2017-12-19T20:58:15.0Z,2019-09-17T19:02:03.0Z,2017-12-15 00:00:00+00:00,2022-12-14 23:59:59+00:00
2,21939,2.0,La Maldición de las Hormigas Gigantes,La_Maldicion_de_las,NaN,Película,2016.0,FI,Terror/Comedia,"Criaturas,Plagas,Adolescentes,Fantasía,Video J...",Un grupo de motociclistas busca diversión el d...,Un grupo de motociclistas descubre un laborato...,"Mark Arnold, Harry Lister Smith, Vanessa Grass...",Marko Mäkilaakso,82.0,Juvenil,Y,N,N,N,N,N,N,N,N,N,2018-02-16T13:51:07.0Z,2020-04-28T14:16:38.0Z,2018-01-25 00:00:00+00:00,2020-12-01 23:59:59+00:00
3,9005,3.0,Una Mujer Fantástica,Una_Mujer_Fantastic,NaN,Película,2017.0,CL,Drama,"LGBT,Mujeres,Latinoamérica",Marina y Orlando están enamorados y planean un...,Marina y Orlando están enamorados. Pero cuando...,"Daniela Vega, Francisco Reyes, Luis Gnecco, Am...",Sebastián Lelio,99.0,General,N,N,N,N,N,N,N,N,Y,N,2018-05-26T11:58:44.0Z,2019-11-15T03:00:23.0Z,2018-05-27 00:00:00+00:00,2021-04-30 23:59:59+00:00
4,7391,4.0,Star Trek,Star_Trek,NaN,Película,2009.0,US,Ciencia Ficción/Aventura,"Fantasía,Galaxia,Futurismo,Aliens,Criaturas",El descarado James T. Kirk trata de estar a la...,James T. Kirk trata de estar a la altura del l...,"Chris Pine, Zachary Quinto, Leonard Nimoy, Eri...",J.J. Abrams,126.0,General,N,N,N,N,N,N,N,Y,N,N,2019-05-03T20:07:24.0Z,2020-04-09T04:37:29.0Z,2019-05-02 00:00:00+00:00,2020-12-31 23:59:59+00:00


Vemos si metadata contiene nulos

In [5]:
metadata.isna().sum()

asset_id                  0
content_id               21
title                     0
reduced_title             0
episode_title          4147
show_type                 4
released_year             0
country_of_origin         4
category                  0
keywords                  2
description               2
reduced_desc              0
cast_first_name        8732
credits_first_name    12554
run_time_min              0
audience                  1
made_for_tv               0
close_caption             0
sex_rating                0
violence_rating           0
language_rating           0
dialog_rating             0
fv_rating                 0
pay_per_view              0
pack_premium_1            0
pack_premium_2            0
create_date               0
modify_date               0
start_vod_date            0
end_vod_date              0
dtype: int64

Filtramos en metadata los contenidos nulos y aquellos cuya duración sean igual a cero

In [6]:
metadata = metadata[~metadata.content_id.isna()]

In [7]:
metadata = metadata[metadata.run_time_min != 0]

¿Qué cantidad de filas y columnas tiene metadata?

In [8]:
metadata.shape

(33009, 30)

¿Qué cantidad de valores diferentes tienen las columnas "content_id" y "asset_id"?

In [9]:
metadata.content_id.nunique()

4364

In [10]:
metadata.asset_id.nunique()

33009

Ahora veamos train

In [11]:
train.head()

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
0,0,90627,STATIONARY,18332.0,2021-02-18 22:52:00,2021-02-18 23:35:00,0
1,0,90627,STATIONARY,24727.0,2021-03-24 23:17:00,2021-03-25 00:01:00,0
2,1,3387,STB,895.0,2021-03-15 10:05:00,2021-03-15 10:23:00,0
3,1,3387,STB,895.0,2021-03-15 10:23:00,2021-03-15 11:18:00,1
4,1,3387,STB,26062.0,2021-03-16 09:24:00,2021-03-16 09:44:00,0


¿Qué cantidad de nulos contiene?

In [12]:
train.isna().sum()

customer_id     0
account_id      0
device_type    29
asset_id       22
tunein          0
tuneout         0
resume          0
dtype: int64

Filtramos los "asset_id" nulos

In [13]:
train = train[~train.asset_id.isna()]

In [14]:
train.isna().sum()

customer_id     0
account_id      0
device_type    29
asset_id        0
tunein          0
tuneout         0
resume          0
dtype: int64

¿Qué cantidad de filas y columnas tiene train?

In [15]:
train.shape

(3657779, 7)

¿Qué cantidad de valores diferentes tienen las columnas "account_id" y "asset_id"?

In [16]:
train.account_id.nunique()

113879

In [17]:
train.asset_id.nunique()

23987

A continuación haremos un merge entre ambos datasets, trayéndonos a train las columnas necesarias del dataset de metadata. Seguidamente se eliminarán columnas que bajo nuestro enfoque resultan innecesarias.

In [18]:
new_train = train.merge(metadata[['content_id','asset_id','run_time_min','start_vod_date','end_vod_date']], how='left', on='asset_id')
new_train= new_train.drop(['customer_id','device_type','resume'], axis =1)
new_train.head()

,account_id,asset_id,tunein,tuneout,content_id,run_time_min,start_vod_date,end_vod_date
0,90627,18332.0,2021-02-18 22:52:00,2021-02-18 23:35:00,2040.0,43.0,2021-02-18 00:00:00+00:00,2021-06-30 23:59:59+00:00
1,90627,24727.0,2021-03-24 23:17:00,2021-03-25 00:01:00,2040.0,42.0,2021-03-18 00:00:00+00:00,2021-06-30 23:59:59+00:00
2,3387,895.0,2021-03-15 10:05:00,2021-03-15 10:23:00,1983.0,54.0,2021-03-08 00:00:00+00:00,2021-03-28 23:59:00+00:00
3,3387,895.0,2021-03-15 10:23:00,2021-03-15 11:18:00,1983.0,54.0,2021-03-08 00:00:00+00:00,2021-03-28 23:59:00+00:00
4,3387,26062.0,2021-03-16 09:24:00,2021-03-16 09:44:00,729.0,49.0,2021-03-08 00:00:00+00:00,2021-03-28 23:59:00+00:00


¿Qué cantidad de filas y columnas tiene el nuevo dataframe?

In [19]:
new_train.shape

(3657779, 8)

¿Tiene nulos?

In [20]:
new_train.isna().sum()

account_id          0
asset_id            0
tunein              0
tuneout             0
content_id        430
run_time_min      430
start_vod_date    430
end_vod_date      430
dtype: int64

Crearemos una nueva columna "watching_time" obtenida de la diferencia entre las columnas "tuneout" y "tunein"

In [21]:
new_train['watching_time'] = new_train['tuneout'] - new_train['tunein']
new_train['watching_time'] = new_train['watching_time'].apply(lambda x: x.total_seconds() / 60)
new_train.head()

,account_id,asset_id,tunein,tuneout,content_id,run_time_min,start_vod_date,end_vod_date,watching_time
0,90627,18332.0,2021-02-18 22:52:00,2021-02-18 23:35:00,2040.0,43.0,2021-02-18 00:00:00+00:00,2021-06-30 23:59:59+00:00,43.0
1,90627,24727.0,2021-03-24 23:17:00,2021-03-25 00:01:00,2040.0,42.0,2021-03-18 00:00:00+00:00,2021-06-30 23:59:59+00:00,44.0
2,3387,895.0,2021-03-15 10:05:00,2021-03-15 10:23:00,1983.0,54.0,2021-03-08 00:00:00+00:00,2021-03-28 23:59:00+00:00,18.0
3,3387,895.0,2021-03-15 10:23:00,2021-03-15 11:18:00,1983.0,54.0,2021-03-08 00:00:00+00:00,2021-03-28 23:59:00+00:00,55.0
4,3387,26062.0,2021-03-16 09:24:00,2021-03-16 09:44:00,729.0,49.0,2021-03-08 00:00:00+00:00,2021-03-28 23:59:00+00:00,20.0


Seguidamente convertiremos las columnas "start_vod_date" y "end_vod_date" a tipo fecha y les cambiaremos el formato

In [22]:
new_train.start_vod_date.dtype

datetime64[ns, UTC]

In [23]:
new_train["start_vod_date"] = new_train["start_vod_date"].apply(lambda x: x.replace(tzinfo=None).date())
new_train["end_vod_date"] = new_train["end_vod_date"].apply(lambda x: x.replace(tzinfo=None).date())

In [24]:
new_train.end_vod_date.dtype

dtype('O')

In [25]:
new_train["start_vod_date"] = pd.to_datetime(new_train["start_vod_date"], format='%Y/%m/%d')
new_train["end_vod_date"] = pd.to_datetime(new_train["end_vod_date"], format='%Y/%m/%d')

In [26]:
new_train.end_vod_date.dtype

dtype('<M8[ns]')

In [27]:
new_train.head()

,account_id,asset_id,tunein,tuneout,content_id,run_time_min,start_vod_date,end_vod_date,watching_time
0,90627,18332.0,2021-02-18 22:52:00,2021-02-18 23:35:00,2040.0,43.0,2021-02-18,2021-06-30,43.0
1,90627,24727.0,2021-03-24 23:17:00,2021-03-25 00:01:00,2040.0,42.0,2021-03-18,2021-06-30,44.0
2,3387,895.0,2021-03-15 10:05:00,2021-03-15 10:23:00,1983.0,54.0,2021-03-08,2021-03-28,18.0
3,3387,895.0,2021-03-15 10:23:00,2021-03-15 11:18:00,1983.0,54.0,2021-03-08,2021-03-28,55.0
4,3387,26062.0,2021-03-16 09:24:00,2021-03-16 09:44:00,729.0,49.0,2021-03-08,2021-03-28,20.0


Lo que haremos, a continuación, es sumar el tiempo visto agrupando por "account_id" y por "asset_id"

In [28]:
new_train = new_train.groupby(by=['account_id','asset_id'],as_index=False).agg({'content_id':'max','run_time_min':'max','watching_time':'sum','start_vod_date':'max','end_vod_date':'max'})
new_train.head()

,account_id,asset_id,content_id,run_time_min,watching_time,start_vod_date,end_vod_date
0,0,6397.0,3845.0,43.0,48.0,2021-02-01,2021-06-01
1,0,13056.0,2866.0,86.0,65.0,2020-12-01,2021-12-31
2,0,15900.0,1503.0,128.0,128.0,2020-12-29,2021-03-05
3,0,29811.0,3438.0,79.0,79.0,2020-12-06,2021-05-11
4,0,29897.0,3498.0,96.0,16.0,2020-11-01,2023-10-01


Filtramos interacciones que nos interesan. Vió por lo menos el 40% del contenido

In [29]:
new_train = new_train[(new_train['watching_time'] / new_train['run_time_min']) > 0.40]

Contamos la cantidad de veces que se vieron los distintos contenidos

In [30]:
interactions = new_train.groupby(["account_id", "content_id","start_vod_date","end_vod_date"],as_index= False)["watching_time"].count()
interactions.columns = ["account_id", "content_id","start_vod_date","end_vod_date","count_of_views"]
interactions.head()

,account_id,content_id,start_vod_date,end_vod_date,count_of_views
0,0,1503.0,2020-12-29,2021-03-05,1
1,0,2866.0,2020-12-01,2021-12-31,1
2,0,3438.0,2020-12-06,2021-05-11,1
3,0,3845.0,2021-02-01,2021-06-01,1
4,1,1020.0,2020-12-31,2021-02-18,1


Rango de la cantidad de visualizaciones de los contenidos por "account_id"

In [31]:
interactions.count_of_views.min()

1

In [32]:
interactions.count_of_views.max()

128

Reemplazaremos la cantidad de visualizaciones de los distintos contenidos (por "account_id") por cinco intérvalos que se corresponderán con cinco ratings

In [33]:
def Ratings(df:pd.DataFrame, columna:str):
  df.loc[df[columna] < 25.6, ['ratings']] = 1
  df.loc[df[columna].between(25.6,51.2), ['ratings']] = 2
  df.loc[df[columna].between(51.2,76.8), ['ratings']] = 3
  df.loc[df[columna].between(76.8,102.4), ['ratings']] = 4
  df.loc[df[columna].between(102.4,128), ['ratings']] = 5

In [34]:
Ratings(interactions, 'count_of_views')

In [35]:
interactions.drop('count_of_views', axis=1, inplace=True)

In [36]:
interactions.head()

,account_id,content_id,start_vod_date,end_vod_date,ratings
0,0,1503.0,2020-12-29,2021-03-05,1.0
1,0,2866.0,2020-12-01,2021-12-31,1.0
2,0,3438.0,2020-12-06,2021-05-11,1.0
3,0,3845.0,2021-02-01,2021-06-01,1.0
4,1,1020.0,2020-12-31,2021-02-18,1.0


Veamos la cantidad de valores que posee la columna "ratings" en sus cinco intérvalos

In [37]:
interactions.ratings.value_counts()

1.0    1093342
2.0       2011
3.0        179
4.0         22
5.0         12
Name: ratings, dtype: int64

Con el fin de separar en train y test, ahora analizaremos el rango de fechas

In [38]:
interactions.start_vod_date.min()

Timestamp('2001-08-02 00:00:00')

In [39]:
interactions.start_vod_date.max()

Timestamp('2021-09-14 00:00:00')

In [40]:
interactions_train = interactions[interactions.start_vod_date < datetime(year=2021, month=3, day=1)]
interactions_test = interactions[interactions.start_vod_date >= datetime(year=2021, month=3, day=1)]

¿Qué cantidad de filas y columnas tiene "interactions_train"?

In [41]:
interactions_train.shape

(992906, 5)

¿Y "interactions_test"?

In [42]:
interactions_test.shape

(102660, 5)

Ahora armaremos nuestra matriz de interacciones y seguidamente completaremos los NaN con ceros

In [43]:
matrix_interactions = pd.pivot_table(interactions_train[['account_id','content_id','ratings']], index='account_id', columns='content_id', values='ratings')
matrix_interactions = matrix_interactions.fillna(0)
matrix_interactions.sample(10)

content_id,0.0,1.0,3.0,6.0,7.0,8.0,9.0,12.0,13.0,14.0,18.0,19.0,20.0,22.0,23.0,24.0,26.0,33.0,36.0,37.0,38.0,39.0,40.0,43.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,...,4326.0,4327.0,4328.0,4329.0,4330.0,4331.0,4332.0,4333.0,4334.0,4335.0,4336.0,4337.0,4338.0,4339.0,4340.0,4341.0,4343.0,4344.0,4345.0,4346.0,4347.0,4348.0,4349.0,4350.0,4351.0,4352.0,4353.0,4354.0,4355.0,4356.0,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
90466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


¿Cuál es su forma?

In [44]:
matrix_interactions.shape

(99482, 3681)

A continuación se definirán dos diccionarios que se emplearán posteriormente como referencia

In [45]:
account_id = list(matrix_interactions.index)
account_dict = {}
counter = 0 
for i in account_id:
    account_dict[i] = counter
    counter += 1

In [47]:
content_id = list(matrix_interactions.columns)
content_dict = {}
counter = 0 
for i in content_id:
    content_dict[i] = counter
    counter += 1

Comprimimos la matriz de interacciones dado que es una matriz dispersa

In [48]:
from scipy.sparse import csr_matrix

matrix_interactions_csr = csr_matrix(matrix_interactions.values)

Emplearemos LightFM para generar nuestro modelo. Para ello primero tendremos que instalar la librería, importarla, definirla y luego entrenarla

In [49]:
pip install lightfm

     |████████████████████████████████| 310 kB 5.3 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705352 sha256=003ac1e78193ca082c8627cafb0e6d8b92bd6e9bfb11a555486e2909a387d9f0
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [50]:
from lightfm import LightFM

In [51]:
model = LightFM(random_state=0,
                learning_rate=0.03,
                no_components=100)

model = model.fit(matrix_interactions_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

In [52]:
import numpy as np

Seguidamente se define una función para generar las recomendaciones para un "account_id" en particular

In [54]:
 def recomm(model, interaction, account_id, account_dict, content_dict, nrec_items = 40, show = True):

    n_account, n_content = interaction.shape
    account_x = account_dict[account_id]

    scores = pd.Series(model.predict(account_x,np.arange(n_content)))
    scores.index = interaction.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_content = list(pd.Series(interaction.loc[account_id,:] \
                                [interaction.loc[account_id,:] > 0].index).sort_values(ascending=False))
    
    content_not_available = interactions[~(interactions.end_vod_date > datetime(year=2021, month=3, day=1))].content_id.unique()
    
    #Filtro de contents que ya vió el usuario
    scores = [x for x in scores if x not in known_content]
    #Filtro de contents que no van a estar disponibles a partir de 01/03/2021
    scores = [x for x in scores if x not in content_not_available]
    recomms = scores[0:nrec_items]

    if show == True:
        print ("Account: " + str(account_id))
        print("Known Likes:")
        counter = 1
        for i in known_content:
            print(str(counter) + '- ' + str(i))
            counter+=1

        print("\n Recommended Items:")
        
        counter = 1
        for i in recomms:
            print(str(counter) + '- ' + str(i))
            counter+=1
    return recomms
    

Probamos la función con un account_id al azar

In [55]:
recomms_test = recomm(model, matrix_interactions, 98350, account_dict, content_dict, show = False)
recomms_test 

[2040.0,
 3806.0,
 2160.0,
 116.0,
 2942.0,
 1877.0,
 2627.0,
 3210.0,
 3381.0,
 2012.0,
 3716.0,
 3775.0,
 3712.0,
 3726.0,
 604.0,
 304.0,
 3384.0,
 2091.0,
 3598.0,
 2815.0,
 2043.0,
 1140.0,
 2178.0,
 3711.0,
 292.0,
 3292.0,
 2992.0,
 3091.0,
 36.0,
 1983.0,
 580.0,
 729.0,
 1971.0,
 4362.0,
 3752.0,
 3382.0,
 3026.0,
 2827.0,
 3572.0,
 3035.0]

Podemos ver ahora, por ejemplo, los títulos de los contenidos que le recomendamos

In [56]:
metadata.loc[metadata.content_id.isin(recomms_test), "title"]

142            Madagascar 3: Los fugitivos
341                  T:1 Ep:04 The Undoing
1439          T:1 Ep:01 Stand Up Argentina
1440          T:1 Ep:02 Stand Up Argentina
1441          T:1 Ep:03 Stand Up Argentina
                       ...                
31967                 T:5 Ep:10 This is Us
31988            T:17 Ep:06 Grey's Anatomy
32078    Las aventuras del doctor Dolittle
32667                 T:5 Ep:11 This is Us
33036                       Los sonámbulos
Name: title, Length: 799, dtype: object

Generamos recomendaciones en base a los contenidos más populares para los account_id nuevos (que no figuraban cuando se entrenó el modelo, es decir, luego del 1 de marzo de 2021)

In [57]:
cold_start_recomms = interactions_train\
                      .groupby("content_id")\
                      .agg({"ratings": "mean"})\
                      .sort_values(by="ratings", ascending=False)\
                      .head(10)\
                      .index\
                      .to_list()

In [58]:
cold_start_recomms

[2299.0, 2167.0, 2016.0, 387.0, 1021.0, 2160.0, 2180.0, 2617.0, 1611.0, 2174.0]

Finalmente generamos las recomendaciones para todos los account_id (tanto nuevos como aquellos con los que se entrenó el modelo) filtrando los contenidos vistos y aquellos que no estarán disponibles luego de la fecha mencionada

In [59]:
account = interactions.account_id.unique()
non_cold_account = interactions_train.account_id.unique()

print(len(account), len(non_cold_account))

102637 99482


In [60]:
recomms_dict = {
    "account": [],
    "recomms": [],
    "cold": []
}

for acc in account:
  
  # Recomms personalizadas
  if acc in non_cold_account:
    acc_recomms = recomm(model, matrix_interactions, acc, account_dict, content_dict, nrec_items = 40, show = False)
    recomms_dict["account"].append(acc)
    recomms_dict["recomms"].append(acc_recomms)
    recomms_dict["cold"].append(False) #Para luego tener identificados a los usuarios cold
  
  # Recomms cold
  else:
    recomms_dict["account"].append(acc)
    recomms_dict["recomms"].append(cold_start_recomms)
    recomms_dict["cold"].append(True) #Para luego tener identificados a los usuarios cold

A partir del diccionario definido anteriomente (inmediatamente arriba) obtenemos el dataframe con las distintas recomendaciones para cada uno de los account_id

In [61]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df.sample(10)

,account,recomms,cold
18006,19781,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False
42095,46446,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False
52076,57494,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False
87252,96573,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False
14896,16334,"[2040.0, 3900.0, 3806.0, 2160.0, 2942.0, 1877....",False
40836,45047,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False
102368,113526,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False
44148,48690,"[3900.0, 3806.0, 2160.0, 116.0, 2942.0, 1877.0...",False
81772,90497,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False
60243,66569,"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0...",False


In [62]:
recomms_df.account.nunique()

102637

In [63]:
recomms_df.cold.value_counts()

False    99482
True      3155
Name: cold, dtype: int64

Ahora tenemos que comparar nuestras recomendaciones contra lo que los account_id realmente vieron

In [64]:
interactions_test.head()

,account_id,content_id,start_vod_date,end_vod_date,ratings
41,3,2340.0,2021-03-09,2021-07-20,1.0
52,3,3702.0,2021-03-05,2021-06-30,1.0
62,3,4059.0,2021-03-02,2021-07-31,1.0
63,3,4111.0,2021-03-07,2021-06-30,1.0
64,3,4180.0,2021-03-14,2021-06-30,1.0


Primero ordenamos los contenidos según los ratings que dieron los account_id en el set de test.

Luego, agrupamos y armamos un listado de contenidos para cada account_id.
Este es el listado "ideal" contra el que vamos a comparar nuestras recomendaciones.

In [65]:
df= pd.DataFrame(interactions_test[interactions_test.content_id.index.isin(interactions_test.loc[:,"ratings"][interactions_test.loc[:,"ratings"] > 0].index)].content_id)
df["account_id"] = interactions_test["account_id"].copy()
df["ratings"] = interactions_test["ratings"].copy()
df.head()

,content_id,account_id,ratings
41,2340.0,3,1.0
52,3702.0,3,1.0
62,4059.0,3,1.0
63,4111.0,3,1.0
64,4180.0,3,1.0


In [66]:
ideal_recomms = df.sort_values(by=["account_id", "ratings"], ascending=False).groupby(["account_id"], as_index=False).agg({"content_id": "unique"})
ideal_recomms.head()

,account_id,content_id
0,3,"[2340.0, 3702.0, 4059.0, 4111.0, 4180.0]"
1,6,"[3386.0, 4065.0]"
2,13,"[135.0, 4163.0, 4267.0]"
3,14,[1983.0]
4,18,"[4026.0, 4133.0]"


Ahora vamos a medir MAP.
 
Primero que nada, vamos a unir en un mismo dataframe nuestras recomendaciones  con el set ideal.

In [67]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="account_id", right_on="account")[["account_id", "content_id", "recomms"]]
df_map.columns = ["account_id", "ideal", "recomms"]
df_map.head()

,account_id,ideal,recomms
0,3,"[2340.0, 3702.0, 4059.0, 4111.0, 4180.0]","[2040.0, 3806.0, 2160.0, 116.0, 2942.0, 2627.0..."
1,6,"[3386.0, 4065.0]","[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0..."
2,13,"[135.0, 4163.0, 4267.0]","[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0..."
3,14,[1983.0],"[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0..."
4,18,"[4026.0, 4133.0]","[2040.0, 3900.0, 3806.0, 2160.0, 116.0, 2942.0..."


In [68]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for label, pred in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [69]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.0081
